# Imports

In [1]:
# Import utils
import numpy as np
import pandas as pd
import copy
import time
import datetime as dt
from joblib import dump, load, Parallel, delayed
import os
from tqdm import tqdm

# Import Weights Model
from WeightsModel import PreProcessing
from WeightsModel import MaxQFeatureScaler
from WeightsModel import MaxQDemandScaler
from WeightsModel import RandomForestWeightsModel

# Import Weighted SAA Model
from WeightedSAA import WeightedSAA
from WeightedSAA import RobustWeightedSAA
from WeightedSAA import RollingHorizonOptimization

# Import Experiment
from Experiment import Experiment
from Experiment import Evaluation

# Setup the experiment

**Experiment data** consists of four data sets.

- **ID_Data** (pd.DataFrame) stores identifiers (in particular the product (SKU) identifier and the timePeriod (sale_yearweek) identifier)
- **X_Data** (pd.DataFrame) is the 'feature matrix', i.e., each row is a feature vector $x_{j,n}$ of product $j$ at time $n$
- **Y_Data** (pd.DataFrame) is the demand time series data, i.e., each row is a demand observations $d_{j,n}$ of product $j$ at time $n$
- **X_Data_Columns** (pd.DataFrame) provides 'selectors' for local vs. global feature sets

Data is loaded before each experiment using **load_data()**.

We run an experiment for all given products (SKUs) $k=1,...,M$ over a test planning horizon $t=1,...,T$ with $T=13$ for three different cost parameter settings $\{K, u, h, b\}$ that vary the critical ratio ($CR=\frac{b}{b+h}$) of holding and backlogging yielding
- $CR=0.50$: $\{K=100, u=0.5, h=1, b=1\}$
- $CR=0.75$: $\{K=100, u=0.5, h=1, b=3\}$
- $CR=0.90$: $\{K=100, u=0.5, h=1, b=9\}$

Experiments are run for different choices of the look-ahead $\tau=0,...,4$. For robust models, we vary the parameter $e=\{1,3,6,9,12\}$ (multiplier of the in-sample standard deviation of demand) defining product and sample sepcific uncertainty sets.

In [2]:
# Setup the experiment
experiment_setup = dict(

    # Set paths
    path_data = '/home/fesc/DDDInventoryControl/Data',
    path_weightsmodel = '/home/fesc/DDDInventoryControl/Data/WeightsModel/WithModelCV',
    path_results = '/home/fesc/DDDInventoryControl/Data/Results/WithModelCV',
    
    # Weights models
    global_weightsmodel = 'rfwm_global', 
    local_weightsmodel = 'rfwm_local', 

    # Optimization models
    GwSAA = 'GwSAA',
    GwSAAR = 'GwSAAR',
    wSAA = 'wSAA',
    wSAAR = 'wSAAR',
    SAA = 'SAA',
    ExPost = 'ExPost',
    
    # Set identifier of start period of testing horizon
    timePeriodsTestStart = 114,

    # Set product identifiers
    products = range(1,460+1),   # Products (SKUs) k=1,...,M
    
    # Set problem params
    T = 13,             # Planning horizon T
    ts = range(1,13+1), # Periods t=1,...,T of the planning horizon
    taus = [0,1,2,3,4], # Look-aheads tau=0,...,4
    es = [1,3,6,9,12],  # Uncertainty set specifications e=1,...,12
    es = [0.33, 0.67, 1.00, 1.33, 1.67, 2.00, 2.33, 2.67, 3.00],  # ...
       
    # Set cost params
    cost_params = [
        {'CR': 0.50, 'K': 100, 'u': 0.5, 'h': 1, 'b': 1},
        {'CR': 0.75, 'K': 100, 'u': 0.5, 'h': 1, 'b': 3},
        {'CR': 0.90, 'K': 100, 'u': 0.5, 'h': 1, 'b': 9}
    ],
    
    # Set gurobi params
    gurobi_params = {
    
        'LogToConsole': 0, 
        'Threads': 1, 
        'NonConvex': 2, 
        'PSDTol': 1e-3, # 0.1%
        'MIPGap': 1e-3, # 0.1%
        'NumericFocus': 0, 
        'obj_improvement': 1e-3, # 0.1%
        'obj_timeout_sec': 3*60, # 3 min
        'obj_timeout_max_sec': 10*60, # 10 min
        
    },
    
    # Global weights model params
    global_weightsmodel_params = dict(
    
        # Meta parameters
        model_params = {
            'oob_score': True,
            'random_state': 12345,
            'n_jobs': 4,
            'verbose': 0
        },

        # Hyper params search grid
        hyper_params_grid = {
            'n_estimators': [500, 1000],
            'max_depth': [None],
            'min_samples_split': [x for x in range(20, 100, 20)],  
            'min_samples_leaf': [x for x in range(10, 100, 10)],  
            'max_features': [x for x in range(8, 136, 8)],   
            'max_leaf_nodes': [None],
            'min_impurity_decrease': [0.0],
            'bootstrap': [True],
            'max_samples': [0.80, 0.85, 0.90, 0.95, 1.00]
        },    

        # Tuning params
        tuning_params = {     
            'n_iter': 100,
            'scoring': {'MSE': 'neg_mean_squared_error'},
            'return_train_score': True,
            'refit': 'MSE',
            'random_state': 12345,
            'n_jobs': 8,
            'verbose': 0
        },    

        # Tuning using random search
        random_search = True
    ),
    
    # Local weights model params
    local_weightsmodel_params = dict(
    
        # Meta parameters
        model_params = {
            'oob_score': True,
            'random_state': 12345,
            'n_jobs': 1,
            'verbose': 0
        },

        # Hyper params search grid
        hyper_params_grid = {
            'n_estimators': [25, 50],
            'max_depth': [None],
            'min_samples_split': [x for x in range(2, 20, 2)],  
            'min_samples_leaf': [x for x in range(2, 10, 2)],  
            'max_features': [x for x in range(4, 96, 4)],   
            'max_leaf_nodes': [None],
            'min_impurity_decrease': [0.0],
            'bootstrap': [True],
            'max_samples': [0.80, 0.85, 0.90, 0.95, 1.00]
        },    

        # Tuning params
        tuning_params = {     
            'n_iter': 100,
            'scoring': {'MSE': 'neg_mean_squared_error'},
            'return_train_score': True,
            'refit': 'MSE',
            'random_state': 12345,
            'n_jobs': 32,
            'verbose': 0
        },    

        # Tuning using random search
        random_search = True
        
    )
)

# Make all experiment variables visible locally
locals().update(experiment_setup)

# Initialize Experiment
experiment = Experiment(**experiment_setup)

# I. Model selection on training-validation horizon

## Global Training and Samping

In [ ]:
# Load and unpack experiment data
X, y, ids_products, ids_timePeriods, features, features_to_scale, features_to_scale_with = experiment.load_data('global')

In [ ]:
# Select data range for model CV
X, y = X[ids_timePeriods < timePeriodsTestStart], y[ids_timePeriods < timePeriodsTestStart]
ids_products, ids_timePeriods = ids_products[ids_timePeriods < timePeriodsTestStart], ids_timePeriods[ids_timePeriods < timePeriodsTestStart] 

In [ ]:
# Initialize preprocessing module of the weights model
pp = PreProcessing()

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:  
    
    # Status
    print('#### Look-ahead tau='+str(tau)+'...')

    # Preprocess data for global models
    data = pp.preprocess_weightsmodel_data(X, y, ids_products, ids_timePeriods, timePeriodsTestStart-T, tau, T, 
                                           features, features_to_scale, features_to_scale_with, 
                                           X_scaler=MaxQFeatureScaler(q_outlier=0.975), 
                                           y_scaler=MaxQDemandScaler(q_outlier=0.975))

    # Save
    _ = dump(data, path_weightsmodel+'/global_data_for_model_cv_tau'+str(tau)+'.joblib')      

### Tune weights model

In [ ]:
# Make meta params for tuning global weights model available
locals().update(experiment_setup['global_weightsmodel_params'])

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:
       
    # Using t=1 (i.e, data available before start of testing)
    t=1

    # Load preprocessed data (alternatively, data can be preprocessed here) 
    data = load(path_weightsmodel+'/global_data_for_model_cv_tau'+str(tau)+'.joblib')
    
    # Extract preprocessed data
    locals().update(data[t])
    
    # Initialize
    pp = PreProcessing()
    wm = RandomForestWeightsModel(model_params)

    # Scale features and demands
    X_train = pp.scale(X_train, X_scalers, ids_products_train)
    y_train = pp.scale(y_train, y_scalers, ids_products_train)   

    # CV time series splits
    cv_folds = pp.split_timeseries_cv(n_splits=3, ids_timePeriods=ids_timePeriods_train)
    
    # CV search
    cv_results = wm.tune(X_train, y_train, cv_folds, hyper_params_grid, tuning_params, random_search, print_status=True)
    wm.save_cv_result(path=path_weightsmodel+'/'+global_weightsmodel+'_cv_for_model_cv_tau'+str(tau)+'.joblib')

### Fit weight functions and generate weights

In [ ]:
# Set meta params
model_params = {
    'n_jobs': 32,
    'verbose': 0
}

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:

    # Status
    print('#### Look-ahead tau='+str(tau)+'...')
    start_time = dt.datetime.now().replace(microsecond=0)
        
    # Initialize
    weights, weightfunctions_times, weights_times = {}, {}, {}
        
    # For each period t=1,...,T
    for t in ts:
        
        # Load preprocessed data (alternatively, data can be preprocessed here)
        data = load(path_weightsmodel+'/global_data_for_model_cv_tau'+str(tau)+'.joblib')
        
        # Extract preprocessed data
        locals().update(data[t])
            
        # Adjust look-ahead tau to account for end of horizon
        tau_ = min(tau,T-t)

        # Initialize
        pp = PreProcessing()
        wm = RandomForestWeightsModel(model_params)
        
        # Scale features and demands
        X_train, X_test = pp.scale(X_train, X_scalers, ids_products_train), pp.scale(X_test, X_scalers, ids_products_test)
        y_train, y_test = pp.scale(y_train, y_scalers, ids_products_train), pp.scale(y_test, y_scalers, ids_products_test)           
            
        # Load tuned weights model
        wm.load_cv_result(path=path_weightsmodel+'/'+global_weightsmodel+'_cv_for_model_cv_tau'+str(tau_)+'.joblib')
        
        # Fit weight functions  
        st_exec, st_cpu = time.time(), time.process_time() 
        wm.fit(X_train, y_train)
        weightfunctions_times[t] = {'exec_time_sec': time.time()-st_exec, 'cpu_time_sec': time.process_time()-st_cpu}
      
        # Generate weights  
        st_exec, st_cpu = time.time(), time.process_time() 
        weights[t] = wm.apply(X_train, X_test)
        weights_times[t] = {'exec_time_sec': time.time()-st_exec, 'cpu_time_sec': time.process_time()-st_cpu}
        
    # Status
    print('...done in', dt.datetime.now().replace(microsecond=0) - start_time)    
        
    # Save
    _ = dump(weights, path_weightsmodel+'/'+global_weightsmodel+'_weights_for_model_cv_tau'+str(tau)+'.joblib')   
    _ = dump(weightfunctions_times, path_weightsmodel+'/'+global_weightsmodel+'_weightfunctions_times_for_model_cv_tau'+str(tau)+'.joblib') 
    _ = dump(weights_times, path_weightsmodel+'/'+global_weightsmodel+'_weights_times_for_model_cv_tau'+str(tau)+'.joblib')    

## Local Training and Sampling

In [ ]:
# Load and unpack experiment data
X, y, ids_products, ids_timePeriods, features = experiment.load_data('local')

In [ ]:
# Select data range for model CV
X, y = X[ids_timePeriods < timePeriodsTestStart], y[ids_timePeriods < timePeriodsTestStart]
ids_products, ids_timePeriods = ids_products[ids_timePeriods < timePeriodsTestStart], ids_timePeriods[ids_timePeriods < timePeriodsTestStart] 

In [ ]:
# Initialize preprocessing module of the weights model
pp = PreProcessing()

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:  
    
    # Status
    print('#### Look-ahead tau='+str(tau)+'...')

    # Preprocess data for local models
    data = pp.preprocess_weightsmodel_data(X, y, ids_products, ids_timePeriods, timePeriodsTestStart-T, tau, T, products=products)

    # Save
    _ = dump(data, path_weightsmodel+'/local_data_for_model_cv_tau'+str(tau)+'.joblib')      

### Tune weights model

In [ ]:
# Make meta params for tuning global weights model available
locals().update(experiment_setup['local_weightsmodel_params'])

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Status
    print('Look-ahead tau='+str(tau)+'...')
    start_time = dt.datetime.now().replace(microsecond=0)
    
    # Load preprocessed data (alternatively, data can be preprocessed here) 
    data = load(path_weightsmodel+'/local_data_for_model_cv_tau'+str(tau)+'.joblib')
        
    # Initialize
    cv_results = {}
    
    # For each product (SKU) k=1,...,M
    for product in products:

        # Using t=1 (i.e, data available before start of testing)
        t=1

        # Extract preprocessed data
        locals().update(data[product][t])

        # Initialize
        pp = PreProcessing()
        wm = RandomForestWeightsModel(model_params)
        
        # CV time series splits
        cv_folds = pp.split_timeseries_cv(n_splits=3, ids_timePeriods=ids_timePeriods_train)
        
        # CV search
        cv_results[product] = wm.tune(X_train, y_train, cv_folds, hyper_params_grid, tuning_params, random_search, print_status=False)

        # Status
        print('Product '+str(product)+' of '+str(len(products))+' in', dt.datetime.now().replace(microsecond=0) - start_time, end='\r', flush=True)

    # Save
    _ = dump(cv_results, path_weightsmodel+'/'+local_weightsmodel+'_cv_for_model_cv_tau'+str(tau)+'.joblib')
    print('')

### Fit weight functions and generate weights

In [ ]:
# Set meta params
model_params = {
    'n_jobs': 32,
    'verbose': 0
}

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Status
    print('Look-ahead tau='+str(tau)+'...')
    start_time = dt.datetime.now().replace(microsecond=0)
    
    # Initialize
    weights, weightfunctions_times, weights_times = {}, {}, {}
    
    # Load preprocessed data (alternatively, data can be preprocessed here) 
    data = load(path_weightsmodel+'/local_data_for_model_cv_tau'+str(tau)+'.joblib')
    
    # For each product (SKU) k=1,...,M
    for product in products:
        
        # Initialize
        weights[product], weightfunctions_times[product], weights_times[product] = {}, {}, {}
    
        # For each period t=1,...,T
        for t in ts:

            # Extract preprocessed data
            locals().update(data[product][t])
                    
            # Adjust look-ahead tau to account for end of horizon
            tau_ = min(tau,T-t)
            
            # Initialize
            pp = PreProcessing()
            wm = RandomForestWeightsModel(model_params)

            # Load tuned weights model
            wm.load_cv_result(path=path_weightsmodel+'/'+local_weightsmodel+'_cv_for_model_cv_tau'+str(tau_)+'.joblib', product=product)

            
            # Fit weight functions  
            st_exec, st_cpu = time.time(), time.process_time() 
            wm.fit(X_train, y_train)
            weightfunctions_times[product][t] = {'exec_time_sec': time.time()-st_exec, 'cpu_time_sec': time.process_time()-st_cpu}

            # Generate weights  
            st_exec, st_cpu = time.time(), time.process_time() 
            weights[product][t] = wm.apply(X_train, X_test)
            weights_times[product][t] = {'exec_time_sec': time.time()-st_exec, 'cpu_time_sec': time.process_time()-st_cpu}

        # Status
        print('Product '+str(product)+' of '+str(len(products))+' in', dt.datetime.now().replace(microsecond=0) - start_time, end='\r', flush=True) 
        
    # Save
    _ = dump(weights, path_weightsmodel+'/'+local_weightsmodel+'_weights_for_model_cv_tau'+str(tau)+'.joblib')   
    _ = dump(weightfunctions_times, path_weightsmodel+'/'+local_weightsmodel+'_weightfunctions_times_for_model_cv_tau'+str(tau)+'.joblib') 
    _ = dump(weights_times, path_weightsmodel+'/'+local_weightsmodel+'_weights_times_for_model_cv_tau'+str(tau)+'.joblib')    
    print('')

## Rolling Horizon Optimization

### (a) Rolling Horizon Global Weighted SAA (GwSAA)

In [ ]:
# Set number of cores
n_jobs = 32

In [ ]:
# Make all experiment variables visible locally
locals().update(experiment_setup)

In [ ]:
# Set path
if not os.path.exists(path_results+'/'+GwSAA+'_for_model_cv'): os.mkdir(path_results+'/'+GwSAA+'_for_model_cv')

# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Print:
    print('Look-ahead tau='+str(tau)+'...')
    
    # Initialize Experiment
    experiment = Experiment(global_weightsmodel+'_for_model_cv', GwSAA+'_for_model_cv', **experiment_setup)

    # Load preprocessed data (alternatively, data can be preprocessed here)
    data = load(path_weightsmodel+'/global_data_for_model_cv_tau'+str(tau)+'.joblib')
    
    # Load weights
    weights = load(path_weightsmodel+'/'+global_weightsmodel+'_weights_for_model_cv_tau'+str(tau)+'.joblib') 
    
    # Preprocess experiment data
    weights_, samples_, actuals_ = experiment.preprocess_data(data, weights)
    
    # For each product (SKU) k=1,...,M
    with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
        resultslog = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(tau=tau, product=product, wsaamodel=WeightedSAA(), weights=weights_[product], 
                                                                     samples=samples_[product], actuals=actuals_[product]) 
                                             for product in products)

### (b) Rolling Horizon Global Robust Weighted SAA (GwSAA-R)

In [3]:
# Set number of cores
n_jobs = 32

In [4]:
# Make all experiment variables visible locally
locals().update(experiment_setup)

In [5]:
# Set new e values to try
es = [0.33, 0.67, 1.00, 1.33, 1.67, 2.00, 2.33, 2.67, 3.00] 

In [ ]:
# Set path
if not os.path.exists(path_results+'/'+GwSAAR+'_for_model_cv_maxnorm'): os.mkdir(path_results+'/'+GwSAAR+'_for_model_cv_maxnorm')

# For each uncertainty set specification
for e in es:
    
    # Print:
    print('Uncertainty set parameter e='+str(e)+'...')

    # For each look-ahead tau=0,...,4
    for tau in taus:

        # Print:
        print('...look-ahead tau='+str(tau)+'...')
        
        # Initialize Experiment
        experiment = Experiment(global_weightsmodel+'_for_model_cv', GwSAAR+'_for_model_cv_maxnorm', **experiment_setup)

        # Load preprocessed data (alternatively, data can be preprocessed here)
        data = load(path_weightsmodel+'/global_data_for_model_cv_tau'+str(tau)+'.joblib')
    
        # Load weights
        weights = load(path_weightsmodel+'/'+global_weightsmodel+'_weights_for_model_cv_tau'+str(tau)+'.joblib') 

        # Preprocess experiment data
        weights_, samples_, actuals_, epsilons_ = experiment.preprocess_data(data, weights, e, maxnorm=True)
        
        # For each product (SKU) k=1,...,M
        with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
            resultslog = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(tau=tau, product=product, e=e, wsaamodel=RobustWeightedSAA(), 
                                                                         weights=weights_[product], samples=samples_[product], 
                                                                         actuals=actuals_[product], epsilons=epsilons_[product]) 
                                                 for product in products)

Uncertainty set parameter e=0.33...
...look-ahead tau=0...


Progress: 100%|██████████| 460/460 [16:13<00:00,  2.12s/it]


...look-ahead tau=1...


Progress: 100%|██████████| 460/460 [1:02:59<00:00,  8.22s/it]


...look-ahead tau=2...


Progress: 100%|██████████| 460/460 [1:31:36<00:00, 11.95s/it]


...look-ahead tau=3...


Progress:   7%|▋         | 34/460 [14:38<2:32:49, 21.52s/it] /home/fesc/.conda/envs/DDDInventoryControl/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
Progress:  97%|█████████▋| 446/460 [2:20:52<04:26, 19.06s/it]  

### (c) Rolling Horizon Local Weighted SAA (wSAA)

In [ ]:
# Set number of cores
n_jobs = 32

In [ ]:
# Make all experiment variables visible locally
locals().update(experiment_setup)

In [ ]:
# Set path
if not os.path.exists(path_results+'/'+wSAA+'_for_model_cv'): os.mkdir(path_results+'/'+wSAA+'_for_model_cv')

# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Print:
    print('Look-ahead tau='+str(tau)+'...')
    
    # Initialize Experiment
    experiment = Experiment(local_weightsmodel+'_for_model_cv', wSAA+'_for_model_cv', **experiment_setup)

    # Load preprocessed data (alternatively, data can be preprocessed here)
    data = load(path_weightsmodel+'/local_data_for_model_cv_tau'+str(tau)+'.joblib')
    
    # Load weights
    weights = load(path_weightsmodel+'/'+local_weightsmodel+'_weights_for_model_cv_tau'+str(tau)+'.joblib') 
    
    # Preprocess experiment data
    weights_, samples_, actuals_ = experiment.preprocess_data(data, weights)
    
    # For each product (SKU) k=1,...,M
    with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
        resultslog = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(tau=tau, product=product, wsaamodel=WeightedSAA(), weights=weights_[product], 
                                                                     samples=samples_[product], actuals=actuals_[product]) for product in products)

### (d) Rolling Horizon Local Robust Weighted SAA (wSAA-R)

In [ ]:
# Set number of cores
n_jobs = 32

In [ ]:
# Make all experiment variables visible locally
locals().update(experiment_setup)

In [ ]:
# Set new e values to try
#es = [x/10 for x in range(1, 30+1, 1)]  
es = [0.33, 0.67, 1.00, 1.33, 1.67, 2.00, 2.33, 2.67, 3.00] 

In [ ]:
# Set path
if not os.path.exists(path_results+'/'+wSAAR+'_for_model_cv_maxnorm'): os.mkdir(path_results+'/'+wSAAR+'_for_model_cv_maxnorm')

# For each uncertainty set specification
for e in es:
    
    # Print:
    print('Uncertainty set parameter e='+str(e)+'...')

    # For each look-ahead tau=0,...,4
    for tau in taus:

        # Print:
        print('...look-ahead tau='+str(tau)+'...')
        
        # Initialize Experiment
        experiment = Experiment(local_weightsmodel+'_for_model_cv', wSAAR+'_for_model_cv_maxnorm', **experiment_setup)

        # Load preprocessed data (alternatively, data can be preprocessed here)
        data = load(path_weightsmodel+'/local_data_for_model_cv_tau'+str(tau)+'.joblib')
    
        # Load weights
        weights = load(path_weightsmodel+'/'+local_weightsmodel+'_weights_for_model_cv_tau'+str(tau)+'.joblib') 

        # Preprocess experiment data
        weights_, samples_, actuals_, epsilons_ = experiment.preprocess_data(data, weights, e, maxnorm=True)
        
        # For each product (SKU) k=1,...,M
        with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
            resultslog = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(tau=tau, product=product, e=e, wsaamodel=RobustWeightedSAA(), 
                                                                         weights=weights_[product], samples=samples_[product], 
                                                                         actuals=actuals_[product], epsilons=epsilons_[product]) 
                                                 for product in products)

### (e) Baseline model: Rolling Horizon Local Weighted SAA (SAA)

In [ ]:
# Set number of cores
n_jobs = 32

In [ ]:
# Make all experiment variables visible locally
locals().update(experiment_setup)

In [ ]:
# Set path
if not os.path.exists(path_results+'/'+SAA+'_for_model_cv'): os.mkdir(path_results+'/'+SAA+'_for_model_cv')

# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Print:
    print('Look-ahead tau='+str(tau)+'...')
    
    # Initialize Experiment
    experiment = Experiment(local_weightsmodel+'_for_model_cv', SAA+'_for_model_cv', **experiment_setup)

    # Load preprocessed data (alternatively, data can be preprocessed here)
    data = load(path_weightsmodel+'/local_data_for_model_cv_tau'+str(tau)+'.joblib')

    # Preprocess experiment data
    samples_, actuals_ = experiment.preprocess_data(data)
    
    # For each product (SKU) k=1,...,M
    with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
        resultslog = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(tau=tau, product=product, wsaamodel=WeightedSAA(), 
                                                                     samples=samples_[product], actuals=actuals_[product]) 
                                             for product in products)

### (f) Ex-post optimal model with deterministic demand

In [ ]:
# Set number of cores
n_jobs = 32

In [ ]:
# Make all experiment variables visible locally
locals().update(experiment_setup)

In [ ]:
# Set path
if not os.path.exists(path_results+'/'+ExPost+'_for_model_cv'): os.mkdir(path_results+'/'+ExPost+'_for_model_cv')


# Initialize Experiment
experiment = Experiment(local_weightsmodel+'_for_model_cv', ExPost+'_for_model_cv', **experiment_setup)

# Load preprocessed data (alternatively, data can be preprocessed here)
data = load(path_weightsmodel+'/local_data_for_model_cv_tau0.joblib')

# Preprocess experiment data
_, actuals = experiment.preprocess_data(data)

# For ex-post clairvoyant, we can directly solve over the full horizon
actuals_ = {}
for product in products:
    d = []
    for t in ts:
        d = d + [actuals[product][t].item()]
    actuals_[product] = np.array(d).reshape(1,len(d))

# For each product (SKU) k=1,...,M
with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
    resultslog = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(product=product, wsaamodel=WeightedSAA(), actuals=actuals_[product]) 
                                         for product in products)

# II. Evaluation and model parameter selection

In [ ]:
# Make all experiment variables visible locally
locals().update(experiment_setup)

# Initialize Evaluation
evaluation = Evaluation()

## Concatenate and aggregate results per model

In [ ]:
# Rolling Horizon Global Weighted SAA (GwSAA)
results = evaluation.concatenate_results(path_results+'/'+GwSAA+'_for_model_cv', GwSAA+'_for_model_cv', products, taus)
results.to_csv(path_results+'/'+GwSAA+'_for_model_cv'+'_results.csv', sep=',', index=False)
results_agg = evaluation.aggregate_results(results, ['CR', 'tau', 'product'])
results_agg.to_csv(path_results+'/'+GwSAA+'_for_model_cv'+'_results_summary.csv', sep=',', index=False)

In [ ]:
# Rolling Horizon Local Weighted SAA (wSAA)
results = evaluation.concatenate_results(path_results+'/'+wSAA+'_for_model_cv', wSAA+'_for_model_cv', products, taus)
results.to_csv(path_results+'/'+wSAA+'_for_model_cv'+'_results.csv', sep=',', index=False)
results_agg = evaluation.aggregate_results(results, ['CR', 'tau', 'product'])
results_agg.to_csv(path_results+'/'+wSAA+'_for_model_cv'+'_results_summary.csv', sep=',', index=False)

In [ ]:
# # Rolling Horizon Global Robust Weighted SAA (GwSAA-R)
# results = evaluation.concatenate_results(path_results+'/'+GwSAAR+'_for_model_cv', GwSAAR+'_for_model_cv', products, taus, es)
# results.to_csv(path_results+'/'+GwSAAR+'_for_model_cv'+'_results.csv', sep=',', index=False)
# results_agg = evaluation.aggregate_results(results, ['CR', 'tau', 'e', 'product'])
# results_agg.to_csv(path_results+'/'+GwSAAR+'_for_model_cv'+'_results_summary.csv', sep=',', index=False)

In [ ]:
# Rolling Horizon Global Robust Weighted SAA (GwSAA-R) - max norm
es = [0.33, 0.67, 1.00, 1.33, 1.67, 2.00, 2.33, 2.67, 3.00] 

with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(es))) as progress_bar:
    results_ = Parallel(n_jobs=len(es))(delayed(evaluation.concatenate_results)(path_results+'/'+GwSAAR+'_for_model_cv_maxnorm', 
                                                                           GwSAAR+'_for_model_cv_maxnorm', 
                                                                           products, taus, [es_]) for es_ in es)
    
results = pd.concat(results_)
results.to_csv(path_results+'/'+GwSAAR+'_for_model_cv_maxnorm'+'_results.csv', sep=',', index=False)
results_agg = evaluation.aggregate_results(results, ['CR', 'tau', 'e', 'product'])
results_agg.to_csv(path_results+'/'+GwSAAR+'_for_model_cv_maxnorm'+'_results_summary.csv', sep=',', index=False)

In [ ]:
# # Rolling Horizon Local Robust Weighted SAA (wSAA-R)
# results = evaluation.concatenate_results(path_results+'/'+wSAAR+'_for_model_cv', wSAAR+'_for_model_cv', products, taus, es)
# results.to_csv(path_results+'/'+wSAAR+'_for_model_cv'+'_results.csv', sep=',', index=False)
# results_agg = evaluation.aggregate_results(results, ['CR', 'tau', 'e', 'product'])
# results_agg.to_csv(path_results+'/'+wSAAR+'_for_model_cv'+'_results_summary.csv', sep=',', index=False)

In [ ]:
# # Rolling Horizon Local Robust Weighted SAA (wSAA-R) - max norm
# es = [x/10 for x in range(1, 30+1, 1)]  
# results = evaluation.concatenate_results(path_results+'/'+wSAAR+'_for_model_cv_maxnorm', wSAAR+'_for_model_cv_maxnorm', products, taus, es)
# results.to_csv(path_results+'/'+wSAAR+'_for_model_cv_maxnorm'+'_results.csv', sep=',', index=False)
# results_agg = evaluation.aggregate_results(results, ['CR', 'tau', 'e', 'product'])
# results_agg.to_csv(path_results+'/'+wSAAR+'_for_model_cv_maxnorm'+'_results_summary.csv', sep=',', index=False)

In [ ]:
# Rolling Horizon Local Robust Weighted SAA (wSAA-R) - max norm
#es = [x/10 for x in range(1, 30+1, 1)]  
es = [0.33, 0.67, 1.00, 1.33, 1.67, 2.00, 2.33, 2.67, 3.00] 

with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(es))) as progress_bar:
    results_ = Parallel(n_jobs=len(es))(delayed(evaluation.concatenate_results)(path_results+'/'+wSAAR+'_for_model_cv_maxnorm', 
                                                                           wSAAR+'_for_model_cv_maxnorm', 
                                                                           products, taus, [es_]) for es_ in es)
    
results = pd.concat(results_)
results.to_csv(path_results+'/'+wSAAR+'_for_model_cv_maxnorm'+'_results.csv', sep=',', index=False)
results_agg = evaluation.aggregate_results(results, ['CR', 'tau', 'e', 'product'])
results_agg.to_csv(path_results+'/'+wSAAR+'_for_model_cv_maxnorm'+'_results_summary.csv', sep=',', index=False)

In [ ]:
# Rolling Horizon Local SAA (SAA)
results = evaluation.concatenate_results(path_results+'/'+SAA+'_for_model_cv', SAA+'_for_model_cv', products, taus)
results.to_csv(path_results+'/'+SAA+'_for_model_cv'+'_results.csv', sep=',', index=False)
results_agg = evaluation.aggregate_results(results, ['CR', 'tau', 'product'])
results_agg.to_csv(path_results+'/'+SAA+'_for_model_cv'+'_results_summary.csv', sep=',', index=False)

In [ ]:
# Ex-post Clairvoyant (ExPost)
results = evaluation.concatenate_results(path_results+'/'+ExPost+'_for_model_cv', ExPost+'_for_model_cv', products, taus=[None])
results.to_csv(path_results+'/'+ExPost+'_for_model_cv'+'_results.csv', sep=',', index=False)
results_agg = evaluation.aggregate_results(results, ['CR', 'product'])
results_agg.to_csv(path_results+'/'+ExPost+'_for_model_cv'+'_results_summary.csv', sep=',', index=False)

## Select best model parameters per model

In [ ]:
# Load aggregated results
results_GwSAA = pd.read_csv(path_results+'/'+GwSAA+'_for_model_cv'+'_results_summary.csv')
results_wSAA = pd.read_csv(path_results+'/'+wSAA+'_for_model_cv'+'_results_summary.csv')
#results_GwSAAR = pd.read_csv(path_results+'/'+GwSAAR+'_for_model_cv'+'_results_summary.csv')
results_GwSAAR = pd.read_csv(path_results+'/'+GwSAAR+'_for_model_cv_maxnorm'+'_results_summary.csv')
#results_wSAAR = pd.read_csv(path_results+'/'+wSAAR+'_for_model_cv'+'_results_summary.csv')
results_wSAAR = pd.read_csv(path_results+'/'+wSAAR+'_for_model_cv_maxnorm'+'_results_summary.csv')
results_SAA = pd.read_csv(path_results+'/'+SAA+'_for_model_cv'+'_results_summary.csv')
results_ExPost = pd.read_csv(path_results+'/'+ExPost+'_for_model_cv'+'_results_summary.csv')

In [ ]:
# Find best tau
results_GwSAA_best = evaluation.best_tau(results_GwSAA, results_ExPost, groupby=['CR', 'product'])
results_wSAA_best = evaluation.best_tau(results_wSAA, results_ExPost, groupby=['CR', 'product'])
results_GwSAAR_best = evaluation.best_tau(results_GwSAAR, results_ExPost, groupby=['CR', 'product', 'e'])
results_wSAAR_best = evaluation.best_tau(results_wSAAR, results_ExPost, groupby=['CR', 'product', 'e'])
results_SAA_best = evaluation.best_tau(results_SAA, results_ExPost, groupby=['CR', 'product'])

In [ ]:
# Find best e given best tau for robust models
results_GwSAAR_best = evaluation.best_e(results_GwSAAR_best, results_ExPost, groupby=['CR', 'product'])
results_wSAAR_best = evaluation.best_e(results_wSAAR_best, results_ExPost, groupby=['CR', 'product'])

# III. Application to out of sample test horizon

## Updated experiment params with optimal tau and e per product

## Global Training and Samping

The two global models (using 'Global Training and Sampling') are **Rolling Horizon Global Weighted SAA (GwSAA)**, which is our model, and **Rolling Horizon Global Robust Weighted SAA (GwSAA-R)**, which is the analogous model with robust extension. Given product $k$, period $t$, and look-ahead $\tau$, both models apply Weighted SAA over the 'global' distribution $\{\{w_{j,t,\tau}^{\,i}(x_{k,t}^{\,i}),(d_{j,t}^{\,i},...,d_{j,t+\tau}^{\,i})\}_{i=1}^{N_{j,t,\tau}}\}_{j=1}^{M}$, with weight functions $w_{j,t,\tau}(\,\cdot\,)$ trained (once for all products) on data $S_{t,\tau}^{\,\text{Global}}=\{\{(x_{j,t}^{\,i},d_{j,t}^{\,i},...,d_{j,t+\tau}^{\,i})\}_{i=1}^{N_{j,t,\tau}}\}_{j=1}^{M}$.

We first load **global** experiment data and then preprocess the data for all look-aheads $\tau=1,...,4$ and periods $t=1,...,T$ upfront. With this, we can later easily load and reuse the data which is needed for several steps along the experiment pipeline. Preprocessing includes reshaping demand time series into $(\tau+1)$-periods rolling look-ahead horizon sequences and mapping corresponding features accordingly. Furthermore, features and demands are scaled for training (and later rescaled for sampling). 

The weights models - and thus the data used, weight functions, and weights per sample - are the same for the two global models **GwSAA** and **GwSAA-R**. First, we tune the hyper parameters of the random forest weights model for each given look-ahead $\tau$ (as for each look-ahead $\tau$ we have a different response for the multi-output random forest regressor). Second, we fit all weight functions (for each look-ahead $\tau=0,...,4$ and over periods $t=1,...,T$) and generate all weights (for each look-ahead $\tau=0,...,4$, over periods $t=1,...,T$, and for each product (SKU) $k=1,...,M$).

In [ ]:
# Load and unpack experiment data
X, y, ids_products, ids_timePeriods, features, features_to_scale, features_to_scale_with = experiment.load_data('global')

In [ ]:
# Initialize preprocessing module of the weights model
pp = PreProcessing()

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:  
    
    # Status
    print('#### Look-ahead tau='+str(tau)+'...')

    # Preprocess data for global models
    data = pp.preprocess_weightsmodel_data(X, y, ids_products, ids_timePeriods, timePeriodsTestStart, tau, T, 
                                           features, features_to_scale, features_to_scale_with, 
                                           X_scaler=MaxQFeatureScaler(q_outlier=0.975), 
                                           y_scaler=MaxQDemandScaler(q_outlier=0.975))

    # Save
    _ = dump(data, path_weightsmodel+'/global_data_tau'+str(tau)+'.joblib')      

### Tune weights model

To tune the hyper parameters of the global random forest weights model, we use 3-fold rolling timeseries cross-validation on the training data and perform random search with 100 iterations over the specified hyper parameter search grid.

In [ ]:
# Make meta params for tuning global weights model available
locals().update(experiment_setup['global_weightsmodel_params'])

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:
       
    # Using t=1 (i.e, data available before start of testing)
    t=1

    # Load preprocessed data (alternatively, data can be preprocessed here) 
    data = load(path_weightsmodel+'/global_data_tau'+str(tau)+'.joblib')
    
    # Extract preprocessed data
    locals().update(data[t])
    
    # Initialize
    pp = PreProcessing()
    wm = RandomForestWeightsModel(model_params)

    # Scale features and demands
    X_train = pp.scale(X_train, X_scalers, ids_products_train)
    y_train = pp.scale(y_train, y_scalers, ids_products_train)   

    # CV time series splits
    cv_folds = pp.split_timeseries_cv(n_splits=3, ids_timePeriods=ids_timePeriods_train)
    
    # CV search
    cv_results = wm.tune(X_train, y_train, cv_folds, hyper_params_grid, tuning_params, random_search, print_status=True)
    wm.save_cv_result(path=path_weightsmodel+'/'+global_weightsmodel+'_cv_tau'+str(tau)+'.joblib')

### Fit weight functions and generate weights

We now fit the global random forest weights model (i.e., the weight functions) for each $\tau=0,...,4$ and over periods $t=1,...,T$. This is done across all products at once (global training). Then, for each $\tau=0,...,4$ and over periods $t=1,...,T$, we generate for each product (SKU) $k=1,...,M$ the weights given the test feature $x_{k,t}$. This is done *jointly* across products (by using $x_{t}=(x_{1,t},...,x_{M,t})^{\top}$) for computational efficiency - the weights for each individual product can be extracted afterwards.

In [ ]:
# Set meta params
model_params = {
    'n_jobs': 32,
    'verbose': 0
}

### UPDATE: we need to fit for all tau but generate weights per product only for optimal tau per product, respectively

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:

    # Status
    print('#### Look-ahead tau='+str(tau)+'...')
    start_time = dt.datetime.now().replace(microsecond=0)
        
    # Initialize
    weights, weightfunctions_times, weights_times = {}, {}, {}
        
    # For each period t=1,...,T
    for t in ts:
        
        # Load preprocessed data (alternatively, data can be preprocessed here)
        data = load(path_weightsmodel+'/global_data_tau'+str(tau)+'.joblib')
        
        # Extract preprocessed data
        locals().update(data[t])
            
        # Adjust look-ahead tau to account for end of horizon
        tau_ = min(tau,T-t)

        # Initialize
        pp = PreProcessing()
        wm = RandomForestWeightsModel(model_params)
        
        # Scale features and demands
        X_train, X_test = pp.scale(X_train, X_scalers, ids_products_train), pp.scale(X_test, X_scalers, ids_products_test)
        y_train, y_test = pp.scale(y_train, y_scalers, ids_products_train), pp.scale(y_test, y_scalers, ids_products_test)           
            
        # Load tuned weights model
        wm.load_cv_result(path=path_weightsmodel+'/'+global_weightsmodel+'_cv_tau'+str(tau_)+'.joblib')
        
        # Fit weight functions  
        st_exec, st_cpu = time.time(), time.process_time() 
        wm.fit(X_train, y_train)
        weightfunctions_times[t] = {'exec_time_sec': time.time()-st_exec, 'cpu_time_sec': time.process_time()-st_cpu}
      
        # Generate weights  
        st_exec, st_cpu = time.time(), time.process_time() 
        weights[t] = wm.apply(X_train, X_test)
        weights_times[t] = {'exec_time_sec': time.time()-st_exec, 'cpu_time_sec': time.process_time()-st_cpu}
        
    # Status
    print('...done in', dt.datetime.now().replace(microsecond=0) - start_time)    
        
    # Save
    _ = dump(weights, path_weightsmodel+'/'+global_weightsmodel+'_weights_tau'+str(tau)+'.joblib')   
    _ = dump(weightfunctions_times, path_weightsmodel+'/'+global_weightsmodel+'_weightfunctions_times_tau'+str(tau)+'.joblib') 
    _ = dump(weights_times, path_weightsmodel+'/'+global_weightsmodel+'_weights_times_tau'+str(tau)+'.joblib')    

## Local Training and Sampling

The two local models (using 'Local Training and Sampling') are **Rolling Horizon Local Weighted SAA (wSAA)**, and **Rolling Horizon Local Robust Weighted SAA (wSAA-R)**, which is the analogous model with robust extension. Given product $k$, period $t$, and look-ahead $\tau$, both models apply Weighted SAA over the 'local' distribution $\{w_{k,t,\tau}^{\,i}(x_{k,t}^{\,i}),(d_{k,t}^{\,i},...,d_{k,t+\tau}^{\,i})\}_{i=1}^{N_{k,t,\tau}}$, with weight functions $w_{k,t,\tau}(\,\cdot\,)$ trained on data $S_{k,t,\tau}^{\,\text{Local}}=\{(x_{k,t}^{\,i},d_{k,t}^{\,i},...,d_{k,t+\tau}^{\,i})\}_{i=1}^{N_{k,t,\tau}}$ for each product $k=1,...,M$ separately.

We first load **local** experiment data and then preprocess the data for all look-aheads $\tau=1,...,4$ and periods $t=1,...,T$ upfront. With this, we can later easily load and reuse the data which is needed for several steps along the experiment pipeline. Preprocessing includes reshaping demand time series into $(\tau+1)$-periods rolling look-ahead horizon sequences and mapping corresponding features accordingly.

The weights model - and thus the data used, weight functions, and weights per sample - are the same for the two local models **wSAA** and **wSAA-R**. First, we tune the hyper parameters of the random forest weights model for each given look-ahead $\tau$ (as for each look-ahead $\tau$ we have a different response for the multi-output random forest regressor) and for each product (SKU) $k=1,...,M$ separately. Second, we fit all weight functions (for each look-ahead $\tau=0,...,4$ and over periods $t=1,...,T$) for each product (SKU) $k=1,...,M$ separately and generate all weights (for each look-ahead $\tau=0,...,4$, over periods $t=1,...,T$, and for each product (SKU) $k=1,...,M$ separatey).

In [ ]:
# Load and unpack experiment data
X, y, ids_products, ids_timePeriods, features = experiment.load_data('local')

In [ ]:
# Initialize preprocessing module of the weights model
pp = PreProcessing()

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:  
    
    # Status
    print('#### Look-ahead tau='+str(tau)+'...')

    # Preprocess data for local models
    data = pp.preprocess_weightsmodel_data(X, y, ids_products, ids_timePeriods, timePeriodsTestStart, tau, T, products=products)

    # Save
    _ = dump(data, path_weightsmodel+'/local_data_tau'+str(tau)+'.joblib')      

### Tune weights model

To tune the hyper parameters of the local random forest weights model for each product (SKU) $k=1,...,M$, we use 3-fold rolling timeseries cross-validation on the training data and perform random search with 100 iterations over the specified hyper parameter search grid.

In [ ]:
# Make meta params for tuning global weights model available
locals().update(experiment_setup['local_weightsmodel_params'])

### UPDATE: Here we now only need to tune for optimal tau

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Status
    print('Look-ahead tau='+str(tau)+'...')
    start_time = dt.datetime.now().replace(microsecond=0)
    
    # Load preprocessed data (alternatively, data can be preprocessed here) 
    data = load(path_weightsmodel+'/local_data_tau'+str(tau)+'.joblib')
        
    # Initialize
    cv_results = {}
    
    # For each product (SKU) k=1,...,M
    for product in products:

        # Using t=1 (i.e, data available before start of testing)
        t=1

        # Extract preprocessed data
        locals().update(data[product][t])

        # Initialize
        pp = PreProcessing()
        wm = RandomForestWeightsModel(model_params)
        
        # CV time series splits
        cv_folds = pp.split_timeseries_cv(n_splits=3, ids_timePeriods=ids_timePeriods_train)
        
        # CV search
        cv_results[product] = wm.tune(X_train, y_train, cv_folds, hyper_params_grid, tuning_params, random_search, print_status=False)

        # Status
        print('Product '+str(product)+' of '+str(len(products))+' in', dt.datetime.now().replace(microsecond=0) - start_time, end='\r', flush=True)

    # Save
    _ = dump(cv_results, path_weightsmodel+'/'+local_weightsmodel+'_cv_tau'+str(tau)+'.joblib')
    print('')

### Fit weight functions and generate weights

We now fit a local random forest weights model (i.e., the weight functions) for each $\tau=0,...,4$, period $t=1,...,T$, and product (SKU) $k=1,...,M$ separately (local training). Then, for each $\tau=0,...,4$, period $t=1,...,T$, and product (SKU) $k=1,...,M$ separately, we generate the weights given the test feature $x_{k,t}$. This is done *separately* for each product (SKU) $k=1,...,M$.

In [ ]:
# Set meta params
model_params = {
    'n_jobs': 32,
    'verbose': 0
}

### UPDATE: Here we only need to fit for optimal tau

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Status
    print('Look-ahead tau='+str(tau)+'...')
    start_time = dt.datetime.now().replace(microsecond=0)
    
    # Initialize
    weights, weightfunctions_times, weights_times = {}, {}, {}
    
    # Load preprocessed data (alternatively, data can be preprocessed here) 
    data = load(path_weightsmodel+'/local_data_tau'+str(tau)+'.joblib')
    
    # For each product (SKU) k=1,...,M
    for product in products:
        
        # Initialize
        weights[product], weightfunctions_times[product], weights_times[product] = {}, {}, {}
    
        # For each period t=1,...,T
        for t in ts:

            # Extract preprocessed data
            locals().update(data[product][t])
                    
            # Adjust look-ahead tau to account for end of horizon
            tau_ = min(tau,T-t)
            
            # Initialize
            pp = PreProcessing()
            wm = RandomForestWeightsModel(model_params)

            # Load tuned weights model
            wm.load_cv_result(path=path_weightsmodel+'/'+local_weightsmodel+'_cv_tau'+str(tau_)+'.joblib', product=product)

            
            # Fit weight functions  
            st_exec, st_cpu = time.time(), time.process_time() 
            wm.fit(X_train, y_train)
            weightfunctions_times[product][t] = {'exec_time_sec': time.time()-st_exec, 'cpu_time_sec': time.process_time()-st_cpu}

            # Generate weights  
            st_exec, st_cpu = time.time(), time.process_time() 
            weights[product][t] = wm.apply(X_train, X_test)
            weights_times[product][t] = {'exec_time_sec': time.time()-st_exec, 'cpu_time_sec': time.process_time()-st_cpu}

        # Status
        print('Product '+str(product)+' of '+str(len(products))+' in', dt.datetime.now().replace(microsecond=0) - start_time, end='\r', flush=True) 
        
    # Save
    _ = dump(weights, path_weightsmodel+'/'+local_weightsmodel+'_weights_tau'+str(tau)+'.joblib')   
    _ = dump(weightfunctions_times, path_weightsmodel+'/'+local_weightsmodel+'_weightfunctions_times_tau'+str(tau)+'.joblib') 
    _ = dump(weights_times, path_weightsmodel+'/'+local_weightsmodel+'_weights_times_tau'+str(tau)+'.joblib')    
    print('')

## Rolling Horizon Optimization

The rolling horizon optimization approach is best illustrated for the application to GwSAA below. Analogously, this approach is applied for the other models, where the data used for training and sampling, fitting the weights model, and the optimization the model solved, are adjusted accordingly.

**Input:** look-ahead $\tau$, cost parameters $\{K_{k,t},u_{k,t},h_{k,t},b_{k,t}\}_{t=1}^{T}$ for products $k=1,....M$

**For** $t=1,...,T$
    
* $\tau \gets \min\{T-t,\tau\}$     Truncate look-ahead $\tau$ given remaining planning horizon $T-t$
 
* **w.fit**     Train weight functions $w_{j,t,\tau}^{\,i}(\,\cdot\,)$ on data $S_{t,\tau}^{\,\text{Global}}=\{\{(x_{j,t}^{\,i},d_{j,t}^{\,i},...,d_{j,t+\tau}^{\,i})\}_{i=1}^{N_{j,t,\tau}}\}_{j=1}^{M}$
    
* **For** $k=1,...,M$ 
    
    * $x_{k,t} \gets X_{k,t}$     Observe new realization $x_{k,t}$ of the feature vector $X_{k,t}$ for product $k$
        
    * **w.predict**     Apply $x_{k,t}$ to weight functions $w_{j,t,\tau}^{\,i}(\,\cdot\,)$ to retrieve $\{\{w_{j,t,\tau}^{\,i}(x_{k,t})\}_{i=1}^{N_{j,t,\tau}}\}_{j=1}^{M}$
    
    * Get decision $\hat{q}_{k,t}(I_{k,t},x_{k,t})$ by solving **GwSAA**
        \begin{equation*}
            \hat{q}_{k,t}(I_{k,t},x_{k,t}) \in \arg\min_{\substack{q_{t} \in Q_{t} \\ ... \\ q_{t+\tau} \in Q_{t+\tau}}}\sum_{j=1}^{M}\sum_{i=1}^{N_{j,t,\tau}}w_{j,t,\tau}^{\,i}(x_{k,t})c_{k,t,\tau}(q_{t},...,q_{t+\tau},d_{j,t}^{\,i},...,d_{j,t+\tau}^{\,i},I_{k,t})
        \end{equation*}
        
    * $I_{k,t+1} \gets I_{k,t}+\hat{q}_{k,t}(I_{k,t},x_{k,t})-d_{k,t}$     Apply decision $\hat{q}_{k,t}(I_{k,t},x_{k,t})$, observe $d_{k,t} \gets D_{k,t}$
    
    **EndFor**
**EndFor**

### UPDATE: Here we now only need to apply for optimal tau and e, respectively

### (a) Rolling Horizon Global Weighted SAA (GwSAA)

Weighted SAA over "global" distribution $\{\{w_{j,t,\tau}^{\,i}(x_{j,t}^{\,i}),(d_{j,t}^{\,i},...,d_{j,t+\tau}^{\,i})\}_{i=1}^{N_{j,t,\tau}}\}_{j=1}^{M}$, with weight functions $w_{j,t,\tau}(\,\cdot\,)$ trained (once for all products) on data $S_{t,\tau}^{\,\text{Global}}=\{\{(x_{j,t}^{\,i},d_{j,t}^{\,i},...,d_{j,t+\tau}^{\,i})\}_{i=1}^{N_{j,t,\tau}}\}_{j=1}^{M}$. Data is scaled per product for training and rescaled with scaler of the current product applied to all products for sampling.

In [ ]:
# Set number of cores
n_jobs = 64

In [ ]:
# Make all experiment variables visible locally
locals().update(experiment_setup)

In [ ]:
# Set path
if not os.path.exists(path_results+'/'+GwSAA): os.mkdir(path_results+'/'+GwSAA)

# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Print:
    print('Look-ahead tau='+str(tau)+'...')
    
    # Initialize Experiment
    experiment = Experiment(global_weightsmodel, GwSAA, **experiment_setup)

    # Load preprocessed data (alternatively, data can be preprocessed here)
    data = load(path_weightsmodel+'/global_data_tau'+str(tau)+'.joblib')
    
    # Load weights
    weights = load(path_weightsmodel+'/'+global_weightsmodel+'_weights_tau'+str(tau)+'.joblib') 
    
    # Preprocess experiment data
    weights_, samples_, actuals_ = experiment.preprocess_data(data, weights)
    
    # For each product (SKU) k=1,...,M
    with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
        resultslog = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(tau=tau, product=product, wsaamodel=WeightedSAA(), weights=weights_[product], 
                                                                     samples=samples_[product], actuals=actuals_[product]) 
                                             for product in products)

### NEW

In [ ]:
# Set path
if not os.path.exists(path_results+'/'+GwSAA): os.mkdir(path_results+'/'+GwSAA)

# Get optimal taus
tau_ = "list of optimal tau per product"




# Print:
print('Look-ahead tau='+str(tau)+'...')

# Initialize Experiment
experiment = Experiment(global_weightsmodel, GwSAA, **experiment_setup)


### TODO: We now need the data / weights / samples / etc. per product for the optimal tau, respectively ...

# For each look-ahead tau=0,...,4
for tau in taus:

    # Load preprocessed data (alternatively, data can be preprocessed here)
    data = load(path_weightsmodel+'/global_data_tau'+str(tau)+'.joblib')

    # Load weights
    weights = load(path_weightsmodel+'/'+global_weightsmodel+'_weights_tau'+str(tau)+'.joblib') 

    # Preprocess experiment data
    weights_, samples_, actuals_ = experiment.preprocess_data(data, weights)

### 

# For each product (SKU) k=1,...,M
with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
    resultslog = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(tau=tau_[product], product=product, wsaamodel=WeightedSAA(), weights=weights_[product], 
                                                                 samples=samples_[product], actuals=actuals_[product]) 
                                         for product in products)

### (b) Rolling Horizon Global Robust Weighted SAA (GwSAA-R)

Weighted SAA as for GwSAA and minimizing worst case cost over product-specific "global" uncertainty sets $\boldsymbol{\mathcal{U}}_{j,t,\tau}^{\,i}(k)=\{\boldsymbol{\tilde{d}}\in \mathcal{D}_{k,t} \times ... \times \mathcal{D}_{k,t+\tau}: ||\boldsymbol{\tilde{d}}-\boldsymbol{d}_{j}^{\,i}|| \leq \epsilon_{k}\}$, with $\boldsymbol{\tilde{d}}=(\tilde{d}_{t},...,\tilde{d}_{t+\tau})^\top$ and $\boldsymbol{d}_{j}^{\,i}=(d_{j,t}^{\,i},...,d_{j,t+\tau}^{\,i})^\top$, for each pair $(j,i)$.

In [ ]:
# Set number of cores
n_jobs = 32

In [ ]:
# Make all experiment variables visible locally
locals().update(experiment_setup)

In [ ]:
# Set new e values to try
es = [0.33, 0.67, 1.00, 1.33, 1.67, 2.00, 2.33, 2.67, 3.00] 

In [ ]:
# Set path
#if not os.path.exists(path_results+'/'+GwSAAR): os.mkdir(path_results+'/'+GwSAAR)

# For each uncertainty set specification
for e in es:
    
    # Print:
    print('Uncertainty set parameter e='+str(e)+'...')

    # For each look-ahead tau=0,...,4
    for tau in taus:

        # Print:
        print('...look-ahead tau='+str(tau)+'...')

        # Load preprocessed data (alternatively, data can be preprocessed here)
        data = load('/home/fesc/DDDInventoryControl/Data/WeightsModel'+'/global_data_tau'+str(tau)+'.joblib')
    
        # Load weights
        weights = load('/home/fesc/DDDInventoryControl/Data/WeightsModel'+'/'+global_weightsmodel+'_weights_tau'+str(tau)+'.joblib')         
        
        # Preprocess experiment data
        weights_, samples_, actuals_, epsilons_ = experiment.preprocess_data(data, weights, e, maxnorm=True)
        
        for cost_params_ in cost_params:

            products_ = list(results_GwSAAR_best.loc[(results_GwSAAR_best.CR==cost_params_['CR']) & 
                                                     (results_GwSAAR_best.tau==tau) & 
                                                     (results_GwSAAR_best.e==e)]['product'])

            experiment_setup_ = copy.deepcopy(experiment_setup)

            experiment_setup_.update({'cost_params': [cost_params_]})

            # Set path
            if not os.path.exists(path_results+'/'+GwSAAR+'_CR'+str(int(cost_params_['CR']*100))): 
                os.mkdir(path_results+'/'+GwSAAR+'_CR'+str(int(cost_params_['CR']*100)))

            # Initialize Experiment
            experiment = Experiment(global_weightsmodel, GwSAAR+'_CR'+str(int(cost_params_['CR']*100)), **experiment_setup_)
            
            
            # For each product (SKU) k=1,...,M
            with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(products_))) as progress_bar:
                resultslog = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(tau=tau, product=product, e=e, wsaamodel=RobustWeightedSAA(), 
                                                                             weights=weights_[product], samples=samples_[product], 
                                                                             actuals=actuals_[product], epsilons=epsilons_[product]) 
                                                     for product in products_)

### (c) Rolling Horizon Local Weighted SAA (wSAA)

Weighted SAA over "local" distribution $\{w_{k,t,\tau}^{\,i}(x_{k,t}^{\,i}),(d_{k,t}^{\,i},...,d_{k,t+\tau}^{\,i})\}_{i=1}^{N_{k,t,\tau}}$, with weight functions $w_{k,t,\tau}(\,\cdot\,)$ trained (for each product) on data $S_{k,t,\tau}^{\,\text{Local}}=\{(x_{k,t}^{\,i},d_{k,t}^{\,i},...,d_{k,t+\tau}^{\,i})\}_{i=1}^{N_{k,t,\tau}}$.

In [ ]:
# Set number of cores
n_jobs = 64

In [ ]:
# Make all experiment variables visible locally
locals().update(experiment_setup)

In [ ]:
# Set path
if not os.path.exists(path_results+'/'+wSAA): os.mkdir(path_results+'/'+wSAA)

# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Print:
    print('Look-ahead tau='+str(tau)+'...')
    
    # Initialize Experiment
    experiment = Experiment(local_weightsmodel, wSAA, **experiment_setup)

    # Load preprocessed data (alternatively, data can be preprocessed here)
    data = load(path_weightsmodel+'/local_data_tau'+str(tau)+'.joblib')
    
    # Load weights
    weights = load(path_weightsmodel+'/'+local_weightsmodel+'_weights_tau'+str(tau)+'.joblib') 
    
    # Preprocess experiment data
    weights_, samples_, actuals_ = experiment.preprocess_data(data, weights)
    
    # For each product (SKU) k=1,...,M
    with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
        resultslog = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(tau=tau, product=product, wsaamodel=WeightedSAA(), weights=weights_[product], 
                                                                     samples=samples_[product], actuals=actuals_[product]) for product in products)

### (d) Rolling Horizon Local Robust Weighted SAA (wSAA-R)

Weighted SAA as for wSAA and minimizing worst case cost over product-specific "local" uncertainty sets $\boldsymbol{\mathcal{U}}_{k,t,\tau}^{\,i}=\{\boldsymbol{\tilde{d}}\in \mathcal{D}_{k,t} \times ... \times \mathcal{D}_{k,t+\tau}: ||\boldsymbol{\tilde{d}}-\boldsymbol{d}_{k}^{\,i}|| \leq \epsilon_{k}\}$, with $\boldsymbol{\tilde{d}}=(\tilde{d}_{t},...,\tilde{d}_{t+\tau})^\top$ and $\boldsymbol{d}_{k}^{\,i}=(d_{k,t}^{\,i},...,d_{k,t+\tau}^{\,i})^\top$, for each pair $(j,i)$.

In [ ]:
# Set number of cores
n_jobs = 96

In [ ]:
# Make all experiment variables visible locally
locals().update(experiment_setup)

In [ ]:
# Set path
#if not os.path.exists(path_results+'/'+wSAAR): os.mkdir(path_results+'/'+wSAAR)

# For each uncertainty set specification
es = [x/10 for x in range(1, 30+1, 1)]  
for e in es:
    
    # Print:
    print('Uncertainty set parameter e='+str(e)+'...')

    # For each look-ahead tau=0,...,4
    for tau in taus:

        # Print:
        print('...look-ahead tau='+str(tau)+'...')
               
        # Load preprocessed data (alternatively, data can be preprocessed here)
        data = load('/home/fesc/DDDInventoryControl/Data/WeightsModel'+'/local_data_tau'+str(tau)+'.joblib')
    
        # Load weights
        weights = load('/home/fesc/DDDInventoryControl/Data/WeightsModel'+'/'+local_weightsmodel+'_weights_tau'+str(tau)+'.joblib') 

        # Preprocess experiment data
        weights_, samples_, actuals_, epsilons_ = experiment.preprocess_data(data, weights, e, maxnorm=True)
        
        
        for cost_params_ in cost_params:

            products_ = list(results_wSAAR_best.loc[(results_wSAAR_best.CR==cost_params_['CR']) & 
                                                    (results_wSAAR_best.tau==tau) & 
                                                    (results_wSAAR_best.e==e)]['product'])

            experiment_setup_ = copy.deepcopy(experiment_setup)

            experiment_setup_.update({'cost_params': [cost_params_]})

            # Set path
            if not os.path.exists(path_results+'/'+wSAAR+'_CR'+str(int(cost_params_['CR']*100))): 
                os.mkdir(path_results+'/'+wSAAR+'_CR'+str(int(cost_params_['CR']*100)))


            # Initialize Experiment
            experiment = Experiment(local_weightsmodel, wSAAR+'_CR'+str(int(cost_params_['CR']*100)), **experiment_setup_)
    

        
            # For each product (SKU) k=1,...,M
            with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(products_))) as progress_bar:
                resultslog = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(tau=tau, product=product, e=e, wsaamodel=RobustWeightedSAA(), 
                                                                             weights=weights_[product], samples=samples_[product], 
                                                                             actuals=actuals_[product], epsilons=epsilons_[product]) 
                                                     for product in products_)

### (e) Baseline model: Rolling Horizon Local Weighted SAA (SAA)

SAA over the distribution $\{\frac{1}{N_{k,t,\tau}},(d_{k,t}^{\,i},...,d_{k,t+\tau}^{\,i})\}_{i=1}^{N_{k,t,\tau}}$

In [ ]:
# Set number of cores
n_jobs = 64

In [ ]:
# Make all experiment variables visible locally
locals().update(experiment_setup)

In [ ]:
# Set path
if not os.path.exists(path_results+'/'+SAA): os.mkdir(path_results+'/'+SAA)

# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Print:
    print('Look-ahead tau='+str(tau)+'...')
    
    # Initialize Experiment
    experiment = Experiment(local_weightsmodel, SAA, **experiment_setup)

    # Load preprocessed data (alternatively, data can be preprocessed here)
    data = load(path_weightsmodel+'/local_data_tau'+str(tau)+'.joblib')

    # Preprocess experiment data
    samples_, actuals_ = experiment.preprocess_data(data)
    
    # For each product (SKU) k=1,...,M
    with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
        resultslog = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(tau=tau, product=product, wsaamodel=WeightedSAA(), 
                                                                     samples=samples_[product], actuals=actuals_[product]) 
                                             for product in products)

### (f) Ex-post optimal model with deterministic demand

Claircoyant optimal model where demand is known upfront.

In [ ]:
# Set number of cores
n_jobs = 64

In [ ]:
# Make all experiment variables visible locally
locals().update(experiment_setup)

In [ ]:
# Set path
if not os.path.exists(path_results+'/'+ExPost): os.mkdir(path_results+'/'+ExPost)


# Initialize Experiment
experiment = Experiment(local_weightsmodel, ExPost, **experiment_setup)

# Load preprocessed data (alternatively, data can be preprocessed here)
data = load(path_weightsmodel+'/local_data_tau0.joblib')

# Preprocess experiment data
_, actuals = experiment.preprocess_data(data)

# For ex-post clairvoyant, we can directly solve over the full horizon
actuals_ = {}
for product in products:
    d = []
    for t in ts:
        d = d + [actuals[product][t].item()]
    actuals_[product] = np.array(d).reshape(1,len(d))

# For each product (SKU) k=1,...,M
with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
    resultslog = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(product=product, wsaamodel=WeightedSAA(), actuals=actuals_[product]) 
                                         for product in products)